In [1]:
# -*- coding: utf-8 -*-
"""
Created on May 31 2021
@author: Te-Yu,Lu
"""

from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm


pd.set_option("display.max_rows", 1000)    #設定最大能顯示1000rows
pd.set_option("display.max_columns", 1000)

# import keras

# 載入 ckiptagger 自訂詞典
User_Dict = {}
with open("KccLy2Dict2021.txt","r", encoding='utf-8-sig') as UDicts:
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 100    # 未給定權重者一律賦予預設值 150                
dictionary = construct_dictionary(User_Dict)

# 載入模型
ws = WS("../data")
# pos = POS("./data")
# ner = NER("./data")

# 載入 StopWord
stopword_list = []
with open("../dict/KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_list.append(stopword.strip())

/Users/Binying/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/Users/Binying/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [3]:
word = pd.read_csv('委員逐字稿.csv',encoding='utf-8') #匯資料
ro = word[word.Name == "羅致政"]
ro.head(1)

,Unnamed: 0,Session,會期,Date,Meeting_Time,Committe,Meet_name,NNN,Name,Location,S_time,E_time,SPEECH_URL,During,VOD_URL,Speech
9327,9327,9,1,2/19/2016,9:00,院會,第9屆第1會期第1次會議,一、19日上午9時至10時進行國是論壇。二、行政院院長提出施政方針及施政報告並備質詢。三、2...,羅致政,新北市第7選舉區,9:40:25,9:43:32,http://ivod.ly.gov.tw/Demand/Speech/86577,0:03:07,http://ivod.ly.gov.tw/Play/VOD/86577/300K,﻿羅委員致政：（9 時40 分）主席、各位同仁。今天是2 月19 日，第9 屆第1 會期的開...


In [4]:
title_ro = ro[['Speech']] #提出會用到的欄位
title_ro.head(2)

,Speech
9327,﻿羅委員致政：（9 時40 分）主席、各位同仁。今天是2 月19 日，第9 屆第1 會期的開...
9497,﻿羅委員致政：主席、各位列席官員、各位同仁。過去我們都是同事，現在只是角色不同，但我們的\n...


In [5]:
ro_list= list(title_ro['Speech'])

ro_str =(','.join('%s' %id for id in ro_list)) 
#print(ro_str)

In [7]:
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced
strCut = ws([ro_str], sentence_segmentation=True,
                 segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                 coerce_dictionary = dictionary)


print("------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------")
stayed_lst= []
for word in strCut[0]:
     # 刪除停用字
    if word not in stopword_list:
        stayed_lst.append(word)

print(" / ".join(stayed_lst))

------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------
﻿ / 羅委員致政 /   / 時 / 40 /   / 分 / 位 / 同仁 / 2 月 / 19 日 / 第9 /   / 屆 / 第1 /   / 會期 / 開議 / 19 / 數字 / 數字 / 91 / 91 / 520 /   / 馬英九 / 政權 / 交替 / 91 / 天 / 數字 / 聽到 / 破百 / 破百 / 退伍 / 守 / 規矩 / 平穩 / 退伍 / 天 / 聽到 / 消息 / 憂慮 / 總統 / 91 /   / 天 / 樣 / 
 / 外交 / 出訪 / 國家 / 說 / 政治 / 義涵 / 國際 / 義涵 / 最近 / 傳出 / 
 / 程度 / 證實 / 總統 / 出訪 / 訊息 / 擔心 / 總統 / 樣 / 外交 / 動
 / 作 / 影響到 / 台灣 / 國際 / 空間 / 外交 / 出訪 / 意義 / 象徵 / 意義 / 
 / 凸顯 / 邦交國 / 關係 / 友邦 / 關係 / 實質 / 意義 / 雙方 / 
 / 經貿 / 合作 / 外交 / 互動 / 提升 / 作用 / 評估 / 任期 / 剩 / 91 /   / 天 / 總統 / 出
 / 訪 / 事 / 想 / 達 / 象徵 / 意義 / 達成 / 實質 / 意義 / 象徵 / 意義 / 天 / 注
 / 意到 / 國家 / 關心 / 新任 / 總統 / 外交 / 動向 / 離任 / 總統 / 太 / 彰
 / 顯 / 國家 / 國際 / 社會 / 政策 / 義涵 / 象徵 / 意義 / 總統 / 出訪 / 時 / 離任 / 總統 / 要
 / 出訪 / 時 / 大部分 / 國家 / 視為 / 兩 / 國 / 關係 / 象徵 / 意義 / 外交 / 動作 / 實質 / 意義 / 說 / 
 / 擔心 / 總統 / 會不會 / 出訪 / 時 / 開出 / 支票 / 總統 / 急 / 主動 / 出訪 / 邦交 / 國
 / 被動 / 安排 / 訪問 / 天下 / 白吃 / 午餐 / 情況 / 國家 / 提出 / 外交 / 
 / 位 / 總統 / 馬 / 總統 / 會不會 / 做出 / 外交 / 實質 / 承諾 / 

In [8]:
# -*- coding: utf-8 -*-
"""
Input: "KCC Data/CkipNewsCisTest-KccDict2020.txt"
Output: "KCC Data/NewsCisTest-KccDict2020-LDA-TopicNum10.txt"
#CkipNewsCisTest-KccDict2020.txt

@author: Johnson
"""
# When numpy version = 1.19.2 => gensim version should downgrade to 3.8.3
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

/Users/Binying/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [9]:
import numpy
stayed_lst_new = [d.split() for d in stayed_lst]
dict = corpora.Dictionary(numpy.asarray(stayed_lst_new))
print("dict = ", dict)
print("------------------------------")

/Users/Binying/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


dict =  Dictionary(16158 unique tokens: ['\ufeff', '羅委員致政', '時', '40', '分']...)
------------------------------


In [10]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in stayed_lst_new]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:5]]
print("======================================================")

corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('\ufeff', 1)
('羅委員致政', 1)
('時', 1)
('40', 1)


In [12]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("3_NewsCisTest-KccDict2020-LDA-TopicNum10.txt", 'w', encoding='utf-8-sig')
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(stayed_lst_new), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)

strTM =  00.171*"樣" + 0.000*"British" + 0.000*"田" + 0.000*"驗證" + 0.000*"石" + 0.000*"認證" + 0.000*"印尼" + 0.000*"館員" + 0.000*"賺錢" + 0.000*"中光" + 0.000*"郵寄" + 0.000*"Office" + 0.000*"崇良" + 0.000*"俊" + 0.000*"百多" + 0.000*"公立醫院" + 0.000*"PCR" + 0.000*"緊急" + 0.000*"軍艦" + 0.000*"婉玉"

strTM =  10.168*"﻿" + 0.000*"中肯" + 0.000*"法官" + 0.000*"庭長" + 0.000*"後難" + 0.000*"浮現" + 0.000*"沒用" + 0.000*"增補" + 0.000*"援外" + 0.000*"改列為" + 0.000*"獨" + 0.000*"熱愛" + 0.000*"源會" + 0.000*"召募" + 0.000*"派出去" + 0.000*"成功嶺" + 0.000*"恬毅" + 0.000*"社會會" + 0.000*"拘束" + 0.000*"蔡崇義"

strTM =  20.000*"浮現" + 0.000*"蔡崇義" + 0.000*"陳師孟" + 0.000*"中肯" + 0.000*"科學" + 0.000*"沒用" + 0.000*"增補" + 0.000*"援外" + 0.000*"改列為" + 0.000*"獨" + 0.000*"熱愛" + 0.000*"源會" + 0.000*"召募" + 0.000*"派出去" + 0.000*"成功嶺" + 0.000*"恬毅" + 0.000*"Christmas" + 0.000*"答出" + 0.000*"楊芳婉" + 0.000*"庭長"

strTM =  30.428*"羅委員致政" + 0.026*"田" + 0.024*"中光" + 0.020*"時" + 0.016*"厚仁" + 0.012*"石" + 0.011*"崇良" + 0.011*"館處" + 0.009*"架" + 0.008*"名稱" + 0.008*"文忠" + 0.007*"冠志" + 0.006